# Web Scraping using Beautiful Soup in Python

### Import the required libraries

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

### The data required is from a Wiki page containing a table. We will use Beautiful Soup to extract the table and insert it into a Pandas Dataframe

In [74]:
#URL given 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

body = soup.find('body')
table = body.find('table', class_ = "wikitable sortable")
table_rows = table.find_all('tr')

data = [] #Stores the Data
column_name = ['Postcode', 'Boroughs', 'Neighborhoods'] #Name of the Columns for the Dataframe

for tr in table_rows:
    td = tr.find_all('td')
    row_data = [tr.text for tr in td]
    data.append(row_data)
    
df = pd.DataFrame(data, columns = column_name)
df.head()

,Postcode,Boroughs,Neighborhoods
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


### We now clean up the data according to the following rules:

* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [79]:
#Drop the first row
df.drop(0, axis = 0, inplace = True) 

#Drop rows were borough is not assigned
df.drop(df[df['Boroughs'] == 'Not assigned'].index, axis = 0, inplace = True) 

#Replace the additional \n in the Neighborhoods column
df['Neighborhoods'] = df['Neighborhoods'].str.replace('\n','')

#Group neighborhoods with the same postal code
df = df.groupby(['Postcode','Boroughs'])['Neighborhoods'].apply(lambda x: "%s" % ', '.join(x)).to_frame().reset_index()

#For 'Not assigned' neighborhoods, the neighborhood will be the same as the borrow
df.loc[(df.Neighborhoods == 'Not assigned'), 'Neighborhoods'] = df['Boroughs']

df.head(5)

,Postcode,Boroughs,Neighborhoods
0,M1E,Scarborough,"Guildwood, Morningside, West Hill"
1,M1G,Scarborough,Woburn
2,M1H,Scarborough,Cedarbrae
3,M1J,Scarborough,Scarborough Village
4,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"


In [80]:
df.shape

(101, 3)